In [ ]:
# Because MobileDet is based out of TensorFlow 1

%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

!git clone --quiet https://github.com/tensorflow/models.git

TensorFlow 1.x selected.
1.15.2


In [ ]:
# Install the TF Object Detection API (TFOD)
%%bash
pip install --upgrade pip
cd models/research
# Compile protos.
protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
cp object_detection/packages/tf1/setup.py .
python -m pip install --use-feature=2020-resolver .

In [ ]:
checkpoint_link = 'http://download.tensorflow.org/models/object_detection/ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19.tar.gz'
folder_name = 'ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19'

!rm -rf folder_name
!wget -q $checkpoint_link -O checkpoints.tar.gz
!tar -xvf checkpoints.tar.gz
!rm checkpoints.tar.gz

In [ ]:
#Export a TFLite compatible model graph
precision = 'uint8'

!python /content/models/research/object_detection/export_tflite_ssd_graph.py \
    --pipeline_config_path=$folder_name/$precision/pipeline.config \
    --trained_checkpoint_prefix=$folder_name/$precision/model.ckpt-400000 \
    --output_directory=$folder_name/$precision \
    --add_postprocessing_op=true
print("======Graph generated========")
!ls -lh $folder_name/$precision/*.pb

In [ ]:
#Initial setup
# %tensorflow_version 2.x
import tensorflow as tf
tf.__version__

'2.3.0'

In [ ]:
#Select model to quantize
import os

model_checkpoint_name = "ssd_mobiledet_edgetpu_coco_uint8"
model_to_be_quantized = 'ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/uint8/tflite_graph.pb'
if os.path.getsize(model_to_be_quantized) > 0:
    print(f"We are quantizing: {model_checkpoint_name}")
else:
    print("Model not found!")

We are quantizing: ssd_mobiledet_edgetpu_coco_uint8


In [ ]:
# Representative dataset:
!wget -q https://github.com/sayakpaul/Adventures-in-TensorFlow-Lite/releases/download/v0.9.0/train_samples_coco.tar.gz
!tar -xvf train_samples_coco.tar.gz
!rm train_samples_coco.tar.gz

rep_ds = tf.data.Dataset.list_files("train_samples/*.jpg")
HEIGHT, WIDTH = 320, 320

def representative_dataset_gen():
    for image_path in rep_ds:
        img = tf.io.read_file(image_path)
        img = tf.io.decode_image(img, channels=3)
        img = tf.image.convert_image_dtype(img, tf.float32)
        resized_img = tf.image.resize(img, (HEIGHT, WIDTH))
        resized_img = resized_img[tf.newaxis, :]
        yield [resized_img]

In [ ]:
#Quantize and serialize (`uint8` are WIP)
original_precision = "uint8"
quantization_strategy = "int8"

converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
    graph_def_file=model_to_be_quantized, 
    input_arrays=['normalized_input_image_tensor'],
    output_arrays=['TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3'],
    input_shapes={'normalized_input_image_tensor': [1, 320, 320, 3]}
)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # ADDED BY ME!!!
converter.allow_custom_ops = True

# https://github.com/ml-gde/e2e-tflite-tutorials/issues/21#issuecomment-692063937
converter.inference_input_type = tf.uint8
converter.quantized_input_stats = {"normalized_input_image_tensor": (128, 128)}
# No additional optimization: https://github.com/ml-gde/e2e-tflite-tutorials/issues/21#issuecomment-692545579

tflite_model = converter.convert()

tflite_filename = model_checkpoint_name + "_" + quantization_strategy + ".tflite"
open(tflite_filename, 'wb').write(tflite_model)
print(f"TFLite model generated with {quantization_strategy}")
!ls -lh $tflite_filename

TFLite model generated with int8
-rw-r--r-- 1 root root 4.2M Nov 25 13:27 ssd_mobiledet_edgetpu_coco_uint8_int8.tflite


In [ ]:
#Install the latest edge TPU compiler
%%bash
curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
sudo apt-get update
sudo apt-get install edgetpu-compiler

In [3]:
#Compile the file
!edgetpu_compiler -s /content/yolov5s-int8.tflite

Edge TPU Compiler version 15.0.340273435

Model compiled successfully in 79 ms.

Input model: /content/yolov5s-int8.tflite
Input size: 7.58MiB
Output model: yolov5s-int8_edgetpu.tflite
Output size: 7.51MiB
On-chip memory used for caching model parameters: 0.00B
On-chip memory remaining for caching model parameters: 8.05MiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 205
Operation log: yolov5s-int8_edgetpu.log

Model successfully compiled but not all operations are supported by the Edge TPU. A percentage of the model will instead run on the CPU, which is slower. If possible, consider updating your model to use only operations supported by the Edge TPU. For details, visit g.co/coral/model-reqs.
Number of operations that will run on Edge TPU: 1
Number of operations that will run on CPU: 204

Operator                       Count      Status

STRIDED_SLICE                  4          Only Strided-Slice with 

In [ ]:
#Download model
from google.colab import files
files.download('/content/ssd_mobiledet_edgetpu_coco_uint8_int8_edgetpu.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>